In [1]:
import base64
import bokeh
import colorsys
import datetime
import glob    
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import os
import palettable
import pandas as pd
import random
import re
import scipy
import scipy.stats as ss
import string as st
import sys
import time
import umap

from anndata import AnnData
from bokeh import io as bio
from bokeh import layouts as blayouts
from bokeh import models as bmodels
from bokeh import plotting as bplt
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.io import output_file, show
from bokeh.io import show, output_notebook, output_file
from bokeh.io.notebook import show_doc, show_app
from bokeh.layouts import widgetbox, row, column
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.models import HoverTool, LassoSelectTool, Slider, Select, ColumnDataSource, PointDrawTool
from bokeh.models.widgets import AutocompleteInput, MultiSelect, TextInput
from bokeh.models.widgets import Button, Toggle, RadioButtonGroup, Div, Panel, Tabs, RadioGroup
from bokeh.models.widgets import RadioButtonGroup
from bokeh.models.widgets import TableColumn, DataTable, CellEditor, TextEditor, DateEditor, TimeEditor, SelectEditor, NumberEditor, PercentEditor, CheckboxEditor, DateFormatter
from bokeh.plotting import figure, ColumnDataSource
from bokeh.server.server import Server

from collections import defaultdict
from colorcet import fire, bgy
from functools import partial

from numpy import seterr,isneginf,array
from scipy.optimize import curve_fit
from shapely.geometry import LineString, Point
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

from MulticoreTSNE import MulticoreTSNE as TSNEm

#from xarray import DataArray
#import fitsne
#import hdbscan

output_notebook()

Loading BokehJS ...

In [ ]:
### Since I use changing servers (sometimes without permissions to set $PYTHONPATH)
### and since the filepaths change (because of docker mounted volumes paths),
### I define a root_folder, find it, and import my shared functions relative to
### this root folder, across all my .py files 
def findmain():
    currentdir = os.getcwd()
    for i in range(25):
        if ('root_folder.py' in os.walk(currentdir).__next__()[2]) == True:
            return currentdir
        currentdir = os.path.dirname(currentdir)

sys.path.append(findmain())

from root_folder import PathsClass, pickle_save, pickle_load, json_save, json_load
Paths = PathsClass()
from LoadingFile import LoaderClass
Loader = LoaderClass(Paths)

%reload_ext autoreload
%autoreload 2

In [3]:
def sort_nicely( l ):
    """ Sort the given list in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort( key=alphanum_key )
    return l


In [4]:
### GET SOME LOGGING GOING ON :)
import os
if os.path.exists("compare-4-log.py"):
    os.remove("compare-4-log.py")
else:
    pass

from bokeh.util import logconfig
logconfig.basicConfig(filename="compare-4-log.py")

In [5]:
def not_nan_tester(a):
    """Pandas and numpy are not always great at casting 
    types when missing np.nan and strings. this function
    helps to sort it out"""
    #if type(a) == float or type(a) == int: return np.isfinite(a);
    try: 
        return np.isfinite(a)
    except:
        pass
    if type(a) == str or type(a) == np.str_: 
        if a != 'nan':
            return True
        else:
            return False
    else: 
        return False

# Load raw data

In [6]:
Loader.create_dataframe_for_day()
design = pd.read_excel(Paths.data.designfile)
design.index = design['Well_ID']

In [8]:
# NOT USING ANYMORE - NOW FILTERED UPSTREAM. BUT LETS KEEP THEM HERE IN CASE I WILL NEED THEM AGAIN.
# ERROR_CORRECTION_GENES = ['ERCC-00002', 'ERCC-00096', 'ERCC-00108', 'ERCC-00111', 'ERCC-00113', 'ERCC-00130']

In [7]:
# if __name__ == '__main__':
#     ### TEST THAT THE LOADING DOES INDEED WORK
#     try:
#         e65 = Loader.df_days['In_vivo']['e6.5']
#         e75 = Loader.df_days['In_vivo']['e7.5']
#         e85 = Loader.df_days['In_vivo']['e8.5']
#         e = pd.concat([e65,e75,e85], axis = 1)

#         print('#    GENE, CELL'); print('e  ' , e.shape); print('e65' , e65.shape); print('e75' , e75.shape); print('e85' , e85.shape)

#         D3 = Loader.df_days['In_vitro']['D3']
#         D4 = Loader.df_days['In_vitro']['D4']
#         D5 = Loader.df_days['In_vitro']['D5']
#         D6 = Loader.df_days['In_vitro']['D6']
#         D = pd.concat([D3,D4,D5,D6], axis = 1)

#         print('\n#    GENE, CELL'); print('D  ' , D.shape); print('D3 ' , D3.shape); print('D4 ' , D4.shape); print('D5 ' , D5.shape); print('D6 ' , D6.shape); 
#     except:
#         raise('Data loading not working. App will therefore also not work')

# Define data to be used

In [9]:
def color_for_labels(labels, cmap = palettable.colorbrewer.diverging.BrBG_11.mpl_colormap):
    if type(labels) == dict:
        cmap = labels['colormap']
        labels = labels['labels']
    CE = LabelEncoder()
    CE.fit(np.unique(labels))
    labels_int = CE.transform(labels) 
    
    n = len(CE.classes_)
    n_fractions_of_1 = [i/(n-1) for i in range(n)]
    rgbs = cmap(n_fractions_of_1)
    hexs = [matplotlib.colors.rgb2hex(rgb[:3]) for rgb in rgbs]    
    
    label_colors = np.array(hexs)[labels_int]
    
    return label_colors

In [10]:
def load_position_files():
    paths = np.array(glob.glob('./Saved_CompareSelections_Single/*.h5ad'))
    filenames = np.array([path.split('/')[-1] for path in paths])
    names = np.array([name.split('{}')[0] for name in filenames])
    dates = np.array([date.split('{}')[-2] for date in filenames])
    idx = np.argsort(dates)[::-1]
    filenames = filenames[idx]
    names = names[idx]
    dates = dates[idx]
    paths = paths[idx]
    return list(filenames), list(dates), list(names), list(paths)

In [ ]:
import scanpy.api as sc
load_position_files()

In [12]:
#genelist = pickle_load('../Annotation/genelist_mgi_2.pickle')


def modules_genes_go(day):
    # Convert my go-annotation and genelist dictionary into lists sorted on module name (NatSort)
    module_list = []
    genenames_list = []
    go_report_list = []
    
    
    levels =           list(genelist[day][1]['module_all'].keys())
    for j in levels:
        n =             len(genelist[day][1]['module_all'][j]['module'])
        for i in range(n):
            module    = str(genelist[day][1]['module_all'][j]['module'][i])
            module_list.append(module)
            genenames =     genelist[day][1]['module_all'][j]['names'][i]
            genenames_list.append(genenames)
            go_report =     genelist[day][1]['module_all'][j]['GO_report'][i]
            go_report_list.append(go_report)
    
    
    module_list = [s.replace('[', '').replace(']', '').strip() for s in module_list]
    module_list_sorted = sort_nicely(list(module_list))
    idx = [module_list.index(s) for s in module_list_sorted]
    
    
    s_module_list    = list(np.array(module_list)[idx])
    s_genenames_list = list(np.array(genenames_list)[idx])
    s_go_report_list = list(np.array(go_report_list)[idx])
    
    
    return s_module_list, s_genenames_list, s_go_report_list

In [13]:
# A_anndata = DATA['A anndata']
# B_anndata = DATA['B anndata']

def overlap_between_vars_in_annadata(A_anndata, B_anndata, preferred_var_key = 'None'):
    
    ### First we find common gene id types between the selections
    A_var_keys = np.array(A_anndata.var_keys())[['ID_' in i for i in A_anndata.var_keys()]]
    B_var_keys = np.array(B_anndata.var_keys())[['ID_' in i for i in B_anndata.var_keys()]]
    #B_var_keys = ['ID_MGI Gene/Markfder', 'ID_Sfdymbol']
    var_keys = list(set(A_var_keys) & set(B_var_keys))

    if len(var_keys) == 0:
        return('Sadness ;-;')
    elif preferred_var_key != 'None' and preferred_var_key not in var_keys:
        return('Sadness ;-;')
    else:
        
        if preferred_var_key in var_keys:
            var_keys = [preferred_var_key]
            
        # Find the gene type IDs present in the two anndata objects
        A_overlap_keys = A_anndata.var.loc[:,var_keys]
        B_overlap_keys = B_anndata.var.loc[:,var_keys]

        # Find which key we can use to identify the biggest overlap between the 
        # This method is annoyingly slow. Look here to improve in future
        counter_key = []
        counter_N = []
        counter_overlaps = []
        
        
        for key in var_keys:
            # Make sure the genes are unique
            A_overlap_keys_unique = A_overlap_keys[key].unique()
            B_overlap_keys_unique = B_overlap_keys[key].unique()
            # Make sure there is no NaNs
            A_overlap_keys_unique_notnan = A_overlap_keys_unique[[not_nan_tester(i) for i in A_overlap_keys_unique]]
            B_overlap_keys_unique_notnan = B_overlap_keys_unique[[not_nan_tester(i) for i in B_overlap_keys_unique]]
            # See overlap in genes for this gene_ID type
            overlaps = np.isin(A_overlap_keys_unique_notnan, B_overlap_keys_unique_notnan)
            overlap = np.sum(overlaps)
            counter_key.append(key)
            counter_N.append(overlap)
            counter_overlaps.append(A_overlap_keys_unique_notnan[overlaps])
        # This it the gene id type and genes that have the overlap
        max_overlap_gene_type = counter_key[np.argmax(counter_N)]
        overlap_genes = counter_overlaps[np.argmax(counter_N)] 
        # Get parts of the two anndata objects with the overlab
        A_anndata_overlap = A_anndata[:,np.isin(A_anndata.var[max_overlap_gene_type], overlap_genes)]
        B_anndata_overlap = B_anndata[:,np.isin(B_anndata.var[max_overlap_gene_type], overlap_genes)]
        # Sort the overlap so we are sure each row will be the same in the two objects for later
        A_var_sortidx = np.argsort(A_anndata_overlap.var[max_overlap_gene_type])
        B_var_sortidx = np.argsort(B_anndata_overlap.var[max_overlap_gene_type])
        # Apply the sorting to the anndata objects
        A_anndata_overlap_sorted = A_anndata_overlap[:, A_var_sortidx]
        B_anndata_overlap_sorted = B_anndata_overlap[:, B_var_sortidx]
        entire_geneset   = set(A_anndata.var[max_overlap_gene_type]) | set(B_anndata.var[max_overlap_gene_type])
        nonoverlap_genes = entire_geneset-set(overlap_genes)
        
        return A_anndata_overlap_sorted, B_anndata_overlap_sorted, overlap_genes, nonoverlap_genes, max_overlap_gene_type

In [14]:
def calculate_statistics(A_anndata_overlap_sorted, B_anndata_overlap_sorted, source3):
    an = A_anndata_overlap_sorted.X
    bn = B_anndata_overlap_sorted.X

    a = an.transpose()
    b = bn.transpose()
    gene        = np.array(A_anndata_overlap_sorted.var['ID_Symbol'])
    a_mean      = a.mean(axis = 1)
    b_mean      = b.mean(axis = 1)
    a_std       = (a.std(axis = 1))
    b_std       = (b.std(axis = 1))
    dif_proc    = (a_mean-b_mean)/((a_mean+b_mean)/2)
    ttest       = np.array([scipy.stats.ttest_ind(a[i], b[i], axis=0, equal_var=False).pvalue for i in range(len(b))])
    fold_change = np.log2(np.array([a_mean,b_mean]).transpose().max(axis = 1) / 
                          (0.000000000000000005 + np.array([a_mean,b_mean]).transpose().min(axis = 1))) * ((np.array(a_mean) > np.array(b_mean)) - 0.5)*2


    total = a_mean+b_mean
    total_std = (total - total.min(axis=0)) / (total.max(axis=0) - total.min(axis=0))
    total_scaled = total_std * (1 - 0.3) + 0.3


    cmap = palettable.colorbrewer.sequential.Greens_9.mpl_colormap
    rgbs = [list(cmap(i)) for i in total_scaled]
    hexs = [matplotlib.colors.rgb2hex(rgb[:3]) for rgb in rgbs]    
    colors = np.array(hexs)

    dummy_data = dict(source3.data)
    dummy_data['fold_change'] = fold_change
    dummy_data['ttest_log10'] = -np.log10(ttest)
    dummy_data['ttest'] = ttest
    dummy_data['dif'] = dif_proc
    dummy_data['colors'] = colors
    dummy_data['total'] = total
    dummy_data['a_std'] = a_std
    dummy_data['b_std'] = b_std
    dummy_data['a_mean'] = a_mean
    dummy_data['b_mean'] = b_mean
    dummy_data['gene'] = gene
    dummy_data['size'] = np.log(total_scaled+0.9)*30

    return dummy_data

In [15]:
def make_thumbnail(anndata):
    x = anndata.uns['coord_tsne_0']
    y = anndata.uns['coord_tsne_1']
    sel = anndata.uns['selection']
    plt.ioff()
    fig = plt.figure(figsize=(3,3))
    plot = plt.plot(x[~sel],y[~sel],'.', color = 'lightgray', markersize = 1)
    plot = plt.plot(x[sel],y[sel],'.', color = 'black', markersize = 1)
    plt.grid(False)
    plt.axis('off')
    filename = anndata.uns['sel_filename']+'.png'
    path_and_filename = './static/' + filename
    plt.savefig(path_and_filename)
    
    with open(path_and_filename, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        imagestring = encoded_string.decode()
    
    return filename, path_and_filename, imagestring

# App

In [16]:
output_notebook()

GI_textlist_holder = [];

#all_gene_names = list(e.index)

DATA = {}
def modify_doc3(doc, DATA = DATA):
    
    global r2

    ##################################### <Plot, label and positions> ###############################################
 
    selector = {'genes': np.array(range(10000))*0}
    current_path = {'path': ''} 
    DATA['A loaded?'] = False
    DATA['B loaded?'] = False

    ### <Plot> ###
    

    lasso = LassoSelectTool(select_every_mousemove = False)
#     hover = HoverTool(tooltips=[("gene", "@gene"),
#                                 ("dif/mean in %", "@dif"),
#                                 ("2 fold change", "@fold_change"),
#                                 ("ttest p", "@ttest"),
#                                 ("mean(a)±std(a)", "@a_mean ± @a_std"), 
#                                 ("mean(b)±std(b)", "@b_mean ± @b_std"), 
#                                 ("sum of means", "@total")])
    custom_hover = HoverTool()
    custom_hover.tooltips = """
    <style>
        .bk-tooltip>div:not(:first-child) {display:none;}
        table {
            font-family: arial, sans-serif;
            border-collapse: collapse;
            width: 100%;
        }

        td, th {
            text-align: left;
            padding: 8px;
        }

        tr:nth-child(even) {
            background-color: @colors;
        }
    </style>
    
    <table>
  <tr>
    <td>Gene</td>
    <td>@gene</td>
  </tr>
  <tr>
    <td>Y2 fold change</td>
    <td>@fold_change</td>
  </tr>
  <tr>
    <td>ttest p</td>
    <td>@ttest</td>
  </tr>
  <tr>
    <td>mean(a)±std(a)</td>
    <td>@a_mean ± @a_std</td>
  </tr>
  <tr>
    <td>mean(b)±std(b)</td>
    <td> @b_mean ± @b_std</td>
  </tr>
  <tr>
    <td>sum of means</td>
    <td>@total</td>
  </tr>
</table>"""
    
    
    fig = bplt.figure(tools=[custom_hover, lasso,"pan,wheel_zoom,box_zoom,reset,box_select, save"], title="Vulcano plot!", plot_width=600, plot_height=600)
#     fig.toolbar.active_inspect = None

    source3 = bmodels.ColumnDataSource(dict(fold_change = [1], #fold_change,
                                            ttest_log10 = [1], #-np.log10(ttest), 
                                            ttest       = [1], #ttest, 
                                            dif         = [1], #dif_proc,
                                            colors      = [1], #colors,
                                            total       = [1], #total,
                                            a_std       = [1], #a_std,
                                            b_std       = [1], #b_std,
                                            a_mean      = [1], #a_mean,
                                            b_mean      = [1], #b_mean,
                                            gene        = [1], #gene,
                                            size        = [20], #np.log(total_scaled+0.9)*30
                                      ));


    r2 = fig.circle(x = 'fold_change', y = 'ttest_log10', color = 'colors', size='size', source = source3)
    fig.xaxis.axis_label = 'b <--- Fold change [log2] ---> a'
    fig.yaxis.axis_label = '-log10(p) [ttest]'
        
    
### </Plot> ###


    position_load_button = Button(label="Load selection from server's disk", button_type="success")
    position_selector = Select(title="Select to load:", value="Load from files first", options=["Load from files first"])
    position_selector_A = Select(title="A) Selection to load:", value="Load from files first", options=["Load from files first"])
    position_selector_B = Select(title="B) Selection to load:", value="Load from files first", options=["Load from files first"])
    imagediv_A = Div(text=""" """, width=216, height=216)
    imagediv_B = Div(text=""" """, width=216, height=216)
    position_info_div = Div(text="""Load the data, might take a few sec.""", width=200, height=100)
    gene_textlist = TextInput(value="", title="Selected genes:")
    
    DATA['position_info_div'] = position_info_div
    
    def selection_handler(attr, old, new, DATA = DATA, source3 = source3, gene_textlist = gene_textlist): # For lasso and other select tools
        ind2 = new #.indices
        if len(ind2) > 0:
            try:
                genes = DATA['source3'].data['gene'] #DATA['overlap_genes']
                #genes = np.array(DATA['anndata'].var['ID_Symbol'])
                selector['genes'] = np.array([i in ind2 for i in range(len(genes))])

            
                gene_textlist.value = ', '.join(genes[selector['genes']])
                gene_textlist.title = 'Selected genes (' + str(len(genes[selector['genes']])) + ') :'
            except:
                pass

    #r2.data_source.on_change('selected', selection_handler)
    r2.data_source.selected.on_change('indices', selection_handler)
    
    def position_load_button_handler(position_selector_A, 
                                     position_selector_B):
        filenames, dates, names, paths = load_position_files()
        position_selector_A.options = ['Select'] + names
        position_selector_B.options = ['Select'] + names
        
    position_load_button.on_click(partial(position_load_button_handler, 
                                          position_selector_A = position_selector_A,
                                          position_selector_B = position_selector_B)) 
    
    

                     
    
    
    def position_selector_handler(attr, old, new, DATA, 
                                  source3, 
                                  position_selector,
                                  A_or_B, 
                                  position_info_div,
                                  imagediv):
        filenames, dates, names, paths = load_position_files()
        if new in names:
            position_selector.title = 'Loading...'
            idx = names.index(new)
            anndata = sc.read(paths[idx])
            
            DATA[A_or_B + ' anndata'] = anndata
            DATA[A_or_B + ' loaded?'] = True
            position_selector.title = A_or_B + ') Select to load:'  
            
            filename, path_and_filename, imagestring = make_thumbnail(anndata)
            imagediv.text = """<img src="data:image/png;base64, """ +imagestring+"""" alt="Red dot" />"""
            
            if DATA['A loaded?'] == True and DATA['B loaded?'] == True:
                ####################
                position_info_div.text = 'Calculating genespace overlap...'
                A_anndata = AnnData(DATA['A anndata'][DATA['A anndata'].uns['selection']])
                B_anndata = AnnData(DATA['B anndata'][DATA['B anndata'].uns['selection']])
                DATA['source3'] = source3
                A_anndata_overlap_sorted,B_anndata_overlap_sorted,overlap_genes, nonoverlap_genes, max_overlap_gene_type = overlap_between_vars_in_annadata(A_anndata, B_anndata, preferred_var_key = 'ID_Symbol')
                DATA['A_anndata_overlap_sorted'] = A_anndata_overlap_sorted
                DATA['B_anndata_overlap_sorted'] = B_anndata_overlap_sorted
                DATA['overlap_genes'] = overlap_genes
                
                position_info_div.text = 'Calculating statistics...'
                source3_dummy_data = calculate_statistics(A_anndata_overlap_sorted, B_anndata_overlap_sorted, source3)
                
                position_info_div.text = 'Updating plot...'
                source3.data = source3_dummy_data
                position_info_div.text = 'Done!'

            
    position_selector_A.on_change('value', partial(position_selector_handler, 
                                                 DATA = DATA,
                                                 source3 = source3,
                                                 position_selector = position_selector_A,
                                                 A_or_B = 'A', 
                                                 position_info_div = position_info_div,
                                                 imagediv = imagediv_A))
                
    position_selector_B.on_change('value', partial(position_selector_handler, 
                                                 DATA = DATA,
                                                 source3 = source3,
                                                 position_selector = position_selector_B,
                                                 A_or_B = 'B',
                                                 position_info_div = position_info_div,
                                                 imagediv = imagediv_B))

    
    doc.add_root(blayouts.layout([[fig,[position_load_button, row([column([position_selector_A, imagediv_A]), column([position_selector_B, imagediv_B])]), position_info_div, gene_textlist]]],sizing_mode='scale_width'))

   

# ### Run server ###
# try: 
#     server.stop()
#     del server
#     del apps
# except:
#     pass
    
    
# apps = {'/app': Application(FunctionHandler(modify_doc3))}


# server = Server(apps, address = '0.0.0.0', allow_websocket_origin = ['*'],  port=10473)
# server.start()

# # # server.stop()
# # # del server

try: del apps
except: pass
try: 
    server1.stop()
    server2.stop()
except: pass
try: del server
except: pass
    
#DATA = {}
    
try:
    apps = {'/app': Application(FunctionHandler(modify_doc3))}    
    #server1 = Server(apps, address = '0.0.0.0', allow_websocket_origin = ['*'],  port=10473)
    #server1.start()
    server2 = Server(apps, address = '0.0.0.0', allow_websocket_origin = ['*'],  port=21073)
    server2.start()
except:
    print('not started')
    try: del apps
    except: print('not del apps')
    try: server.stop()
    except: print('not server stop')
    try: del server
    except: print('not del server')

Loading BokehJS ...

In [17]:
print('Debug statement')

Debug statement


In [18]:
# if __name__ == '__main__':
#     an = DATA['A_anndata_overlap_sorted'].X
#     bn = DATA['B_anndata_overlap_sorted'].X

#     a = an.transpose()
#     b = bn.transpose()
#     gene        = np.array(DATA['A_anndata_overlap_sorted'].var['ID_Symbol'])
#     a_mean      = a.mean(axis = 1)
#     b_mean      = b.mean(axis = 1)
#     a_std       = (a.std(axis = 1))
#     b_std       = (b.std(axis = 1))
#     dif_proc    = (a_mean-b_mean)/((a_mean+b_mean)/2)
#     ttest       = np.array([scipy.stats.ttest_ind(a[i], b[i], axis=0, equal_var=False).pvalue for i in range(len(b))])
#     fold_change = np.log2(np.array([a_mean,b_mean]).transpose().max(axis = 1) / 
#                           (0.000000000000000005 + np.array([a_mean,b_mean]).transpose().min(axis = 1))) * ((np.array(a_mean) > np.array(b_mean)) - 0.5)*2

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log10
